# Advanced RAG 01: Small to Big

### Child-Parent RecursiveRetriever and Sentence Window Retrieval with LlamaIndex

Sources:
- https://docs.llamaindex.ai/en/stable/examples/retrievers/recursive_retriever_nodes.html
- https://docs.llamaindex.ai/en/latest/examples/node_postprocessor/MetadataReplacementDemo.html

In [1]:
! pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "TYPE YOUR API KEY HERE"

In [3]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

--2023-10-27 15:04:51--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 128.84.21.199
Connecting to arxiv.org (arxiv.org)|128.84.21.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘llama2.pdf’

llama2.pdf          100%[===================>]  13.03M  4.53MB/s    in 2.9s    

2023-10-27 15:04:54 (4.53 MB/s) - ‘llama2.pdf’ saved [13661300/13661300]



In [4]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader
from llama_index.response.notebook_utils import display_source_node
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI
import json

In [5]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("llama2.pdf"))

In [6]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [7]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import IndexNode

In [8]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [46]:
node_parser

SimpleNodeParser(text_splitter=SentenceSplitter(chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?', chunking_tokenizer_fn=<function split_by_sentence_tokenizer.<locals>.split at 0x7db46c168af0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7db48a2e6920>, tokenizer=functools.partial(<bound method Encoding.encode of <Encoding 'gpt2'>>, allowed_special='all')), include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7db48a2e6920>)

In [9]:
base_nodes = node_parser.get_nodes_from_documents(docs)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [51]:
base_nodes[-1]

TextNode(id_='node-79', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4fc7fef7-df1d-4913-ac00-a358db22478e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='231b3bd773f012c01b893ada9e009ecf25ee59934614c5a595c9bf1b3a123292'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='90514592-8b40-4297-beae-cae7dd4c36cf', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d32919f6e9eb032a3e15d25d921823a63e2f0ff09a5f981460d67ff08c166ea2')}, hash='fe9ef2e4cd76928824f56f5dae93d26be9ec08619fd3a646b1e53f20ca1c5e67', text='Input Models input text only.\nOutput Models generate text only.\nModel Architecture Llama 2 isanauto-regressivelanguagemodelthatusesanoptimizedtransformer\narchitecture. Thetunedversionsusesupervisedfine-tuning(SFT)andreinforce-\nmentlearning withhuman feedback(RLHF)to aligntohuman preferencesfor\nhelpfulness and safety.\nModel Dates Llama 2 was

In [10]:
from llama_index.embeddings import resolve_embed_model

embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

In [11]:
base_index = VectorStoreIndex(base_nodes, service_context=service_context)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [12]:
retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

In [13]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-22<br>**Similarity:** 0.8486295004749903<br>**Text:** We observe that models
trained from less aggressively filtered pretraining data also required fewer examples to achieve reasonable
safety-alignment. Wereiteratethatthismotivatedchoicedoesimplythatadditionalsafetymitigationsshould
be applied before deployment of base Llama 2 models.
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter). ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view ofmodel capabilities and behaviors that allow us to understand general
patternsinthemodel,buttheydonotprovideafullycomprehensiveviewoftheimpactthemodelmayhave
onpeopleorreal-worldoutcomes;thatwouldrequirestudyofend-to-endproductdeployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender). AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
4.2 Safety Fine-Tuning
In...<br>

**Node ID:** node-65<br>**Similarity:** 0.8475738617788042<br>**Text:** Theassis-
tant’sanswersshouldnotincludeanyharmful,unethical,orsociallybiasedcontent,andshouldmovethe
conversation in a positive direction.
(Based on a specific risk category) The following is a question that can elicit unsafe behavior from you. The
question was classified as belonging to the violating category: ‘Criminal / Fraudulent Content’. You should
answer using the following template:
1. [ifapplicable]Addressimmediatesafetyconcerns. Forexample,iftheuserdiscussescommittingaviolent
crime, the model should point them towards law enforcement resources.
2. Addresstheprompt. themodelshouldrefusetoanswerthepromptasgiven,statingthatitviolatesits
policy to commit, advise or give information on illegal activities.
3. Add context and additional information. If applicable, the model can advise on legal alternatives.
4. Encourageuserstoaskfor/viewadditionalinfoasappropriate. Ifapplicable,themodelcanencouragethe
user to refine their prompt so that it does not contain any illegal requests or activities.
(Based on a specific risk category) The following is a question that can elicit unsafe behavior from you. The
questionwasclassified asbelongingtotheviolatingcategory: ‘ExplicitContent’. Youshould answerusing
the following template:
1. Address immediate safety concerns. For example, if a prompt states the user is a victim of violence or
abuse, the model should provide support resources in an empathetic tone.
2. Address the prompt. the model should refuse to generate explicit sexual o...<br>

In [14]:
query_engine_base = RetrieverQueryEngine.from_args(
    base_retriever, service_context=service_context
)

In [15]:
response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

The key concepts for safety fine-tuning include supervised safety fine-tuning, safety RLHF (Reinforcement Learning from Human Feedback), and safety context distillation. Supervised safety fine-tuning involves gathering adversarial prompts and safe demonstrations to train the model to align with safety guidelines. Safety RLHF integrates safety into the general RLHF pipeline by training a safety-specific reward model and gathering challenging adversarial prompts for fine-tuning. Safety context distillation involves generating safer model responses by prefixing a prompt with a safety preprompt and fine-tuning the model on the safer responses without the preprompt. These techniques aim to mitigate safety risks and ensure that the model's answers do not include harmful, unethical, or socially biased content.


# Chunk References: Smaller Child Chunks Referring to Bigger Parent Chunk

In [16]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(chunk_size=c) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [25]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [52]:
len(all_nodes_dict)

1448

In [55]:
all_nodes_dict

{'630ed4ad-27cc-4a1f-a315-7e7d9bdbe588': IndexNode(id_='630ed4ad-27cc-4a1f-a315-7e7d9bdbe588', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='node-0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='a4e2b17c5d3212c7c9807ab1be88deff6d2fa30d7beb15bdb19e093829955b66'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='631a2469-4720-48cb-95dc-20bc6c890bc1', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='c1cd7a3cd4e6b6f0bf126aec76d521214773696e2cc43ab2e814cffbb76582cf')}, hash='9aedf530bac762f456983dcc676b48da098997325ea380dcb858ba23ab0ddd95', text='Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu

In [18]:
vector_index_chunk = VectorStoreIndex(
    all_nodes, service_context=service_context
)

In [19]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [20]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [21]:
nodes = retriever_chunk.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-22
Retrieving with query id node-22: Can you tell me about the key concepts for safety finetuning


**Node ID:** node-1<br>**Similarity:** 0.873018577198763<br>**Text:** . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
3.4 RLHF Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17
4 Safety 20
4.1 Safety in Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20
4.2 Safety Fine-Tuning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 23
4.3 Red Teaming . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 28
4.4 Safety Evaluation of Llama 2-Chat . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 29
5 Discussion 32
5.1 Learnings and Observations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 32
5.2 Limitations and Ethical Considerations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 34
5.3 Responsible Release Strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 35
6 Related Work 35
7 Conclusion 36
A Appendix 46
A.1 Contributions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 46
A.2 Additional Details for Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 47
A.3 Additional Details for Fine-tuning . . . . . . . . . . . . . . . . . . . . .<br>

**Node ID:** node-22<br>**Similarity:** 0.8656067071505295<br>**Text:** We observe that models
trained from less aggressively filtered pretraining data also required fewer examples to achieve reasonable
safety-alignment. Wereiteratethatthismotivatedchoicedoesimplythatadditionalsafetymitigationsshould
be applied before deployment of base Llama 2 models.
22

TruthfulQA ↑ToxiGen ↓
MPT7B 29.13 22.32
30B 35.25 22.61
Falcon7B 25.95 14.53
40B 40.39 23.44
Llama 17B 27.42 23.00
13B 41.74 23.08
33B 44.19 22.57
65B 48.71 21.77
Llama 27B 33.29 21.25
13B 41.86 26.10
34B 43.45 21.19
70B 50.18 24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentageofgenerationsthatarebothtruthfulandinformative(thehigherthebetter). ForToxiGen,we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view ofmodel capabilities and behaviors that allow us to understand general
patternsinthemodel,buttheydonotprovideafullycomprehensiveviewoftheimpactthemodelmayhave
onpeopleorreal-worldoutcomes;thatwouldrequirestudyofend-to-endproductdeployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender). AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks. Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
Specifically, we use the following techniques in safety fine-tuning:
1.Supervised Safety Fine-Tuning : We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in...<br>

In [22]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk, service_context=service_context
)

In [23]:
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-1
Retrieving with query id node-1: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-22
Retrieving with query id node-22: Can you tell me about the key concepts for safety finetuning
The key concepts for safety fine-tuning include supervised safety fine-tuning, safety RLHF (Reinforcement Learning from Human Feedback), and safety context distillation. 

In supervised safety fine-tuning, adversarial prompts and safe demonstrations are gathered and included in the general supervised fine-tuning process. This helps the model align with safety guidelines even before RLHF and lays the foundation for high-quality human preference data annotation.

In safety RLHF, safety is integrated into the general RLHF pipeline. This involves training a safety-specific reward model and gathering more challenging adversarial prom

# Evaluation

In [73]:
from llama_index.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [74]:
eval_dataset = generate_question_context_pairs(base_nodes)

100%|██████████| 80/80 [02:30<00:00,  1.88s/it]


In [75]:
eval_dataset.save_json("llama2_eval_dataset.json")
# eval_dataset = EmbeddingQAFinetuneDataset.from_json("data/llama2_eval_dataset.json")

In [76]:
import pandas as pd
from llama_index.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)

In [77]:
# base
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)




100%|██████████| 165/165 [00:14<00:00, 11.29it/s]


In [80]:
# chunk
vector_retriever_chunk = vector_index_chunk.as_retriever(
    similarity_top_k=top_k
)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)

results_chunk = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/165 [00:00<?, ?it/s]

Retrieving with query id None: How are the reward models initialized in Llama 2-Chat? Explain the advantage of initializing them from pretrained chat model checkpoints.
Retrieved node with id, entering: node-4
Retrieving with query id node-4: How are the reward models initialized in Llama 2-Chat? Explain the advantage of initializing them from pretrained chat model checkpoints.
Retrieved node with id, entering: node-10
Retrieving with query id node-10: How are the reward models initialized in Llama 2-Chat? Explain the advantage of initializing them from pretrained chat model checkpoints.
Retrieved node with id, entering: node-12
Retrieving with query id node-12: How are the reward models initialized in Llama 2-Chat? Explain the advantage of initializing them from pretrained chat model checkpoints.
Retrieved node with id, entering: node-9
Retrieving with query id node-9: How are the reward models initialized in Llama 2-Chat? Explain the advantage of initializing them from pretrained cha

100%|██████████| 165/165 [00:25<00:00,  6.36it/s]

Retrieved node with id, entering: node-10
Retrieving with query id node-10: How does the reward modeling process work in Llama 2-Chat? Describe the inputs and outputs of the reward model and its role in optimizing the system.
Retrieved node with id, entering: node-9
Retrieving with query id node-9: How does the reward modeling process work in Llama 2-Chat? Describe the inputs and outputs of the reward model and its role in optimizing the system.
Retrieved node with id, entering: node-12
Retrieving with query id node-12: How does the reward modeling process work in Llama 2-Chat? Describe the inputs and outputs of the reward model and its role in optimizing the system.
Retrieved node with id, entering: node-4
Retrieving with query id node-4: How does the reward modeling process work in Llama 2-Chat? Describe the inputs and outputs of the reward model and its role in optimizing the system.
Retrieved node with id, entering: node-30
Retrieving with query id node-30: How does the reward mode

In [101]:
full_results_df = get_retrieval_results_df(
    [
        "Base Retriever",
        "Retriever (Chunk References)"
    ],
    [results_base, results_chunk],
)
display(full_results_df)

,retrievers,hit_rate,mrr
0,Base Retriever,0.769697,0.594644
1,Retriever (Chunk References),0.884848,0.770101


# Sentence Window Retrieval


In [64]:
from llama_index.node_parser import SentenceWindowNodeParser

In [65]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [66]:
node_parser

SentenceWindowNodeParser(sentence_splitter=<function split_by_sentence_tokenizer.<locals>.split at 0x7db37d6c6c20>, window_size=3, window_metadata_key='window', original_text_metadata_key='original_text', include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7db37d5ea3e0>)

In [68]:
sentence_nodes = node_parser.get_nodes_from_documents(docs)

In [69]:
sentence_index = VectorStoreIndex(sentence_nodes, service_context=service_context)

In [102]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)


In [106]:
window_response = query_engine.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(window_response)

The key concepts for safety fine-tuning include supervised safety fine-tuning and safety reinforcement learning from human feedback (RLHF). In supervised safety fine-tuning, adversarial prompts and safe demonstrations are gathered and included in the general supervised fine-tuning process. This helps align the model with safety guidelines even before RLHF and lays the foundation for high-quality human preference data annotation. Safety RLHF involves integrating safety into the general RLHF pipeline. These techniques are used to mitigate safety risks and ensure the safe deployment of the system.


In [95]:
# check the original sentence that was retrieved for each node, as well as the actual window of sentences that was sent to the LLM.
window = sentence_response.source_nodes[0].node.metadata["window"]
sentence = sentence_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed.  For this, it may be necessary to test beyond the groups available in
theBOLDdataset(race,religion,andgender).  AsLLMsareintegratedanddeployed,welookforwardto
continuing research that will amplify their potential for positive impact on these important social issues.
 4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines,andthetechniquesweusetomitigatesafetyrisks.  Weemployaprocesssimilartothegeneral
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
 Specifically, we use the following techniques in safety fine-tuning:
1.Supervised Safety Fine-Tuning : We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in the general supervised fine-tuning process (Sec